In [ ]:
!pip install opendatasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"karmakarabhishek5","key":"36b4a526001d0aea0dbb0308eec30991"}'}

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

ref                                                             title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ------------------------------------------  ------  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                               9KB  2022-10-06 08:55:25           8020        264  1.0              
whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset        11KB  2022-09-30 04:05:11           6467        206  1.0              
whenamancodes/students-performance-in-exams                     Students Performance in Exams                  9KB  2022-09-14 15:14:54          11314        205  1.0              
dimitryzub/walmart-coffee-listings-from-500-stores              Walmart Coffee Listings from 50

In [ ]:
!kaggle datasets download -d uciml/electric-power-consumption-data-set

electric-power-consumption-data-set.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip electric-power-consumption-data-set.zip

Archive:  electric-power-consumption-data-set.zip
replace household_power_consumption.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
pip install elephas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('household_power_consumption.txt', sep=';', 
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')

In [ ]:
len(df)

2075259

In [ ]:
import numpy as np
for cols in df.iloc[:,2:].columns:
    print(cols)
    for x in [cols]:
        q75,q25 = np.percentile(df.loc[:,x],[75,25])
        intr_qr = q75-q25
        
        max1 = q75+(1.5*intr_qr)
        min1 = q25-(1.5*intr_qr)
        
        df.loc[df[x] < min1,x] = np.NaN
        df.loc[df[x] > max1,x] = np.NaN

# sns.heatmap(df.isnull(), yticklabels=False, cmap='viridis')    



df = df.fillna(df.mean())
df.isnull().sum()


df = df[['Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_2', 'Sub_metering_1','Sub_metering_3']]



# We will frame the supervised learning problem as predicting the Global_active_power at the current time (t) 
# given the Global_active_power measurement and other features at the prior time step

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dff = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(-i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i==0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1)) for j in range(n_vars)]        
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        if dropnan:
            agg.dropna(inplace=True)
        return agg

df_resample = df.resample('h').mean() 
df_resample.shape


from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()




# Data spliting into train and test data series. Only 1000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]

# split into input and outputs
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]

# reshape input to be 3D [samples, timesteps, features]
# We reshaped the input into the 3D format as expected by LSTMs, namely [samples, timesteps, features]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


Voltage
Global_intensity
Sub_metering_1
Sub_metering_2
Sub_metering_3


# Global Active Power

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

# Creating Spark context
master_url = "'local[8]'"
conf = SparkConf().setAppName("IOT_LSTM_Elephas_App")
conf.setMaster(master_url)
sc = SparkContext.getOrCreate(conf = conf)

model1 = Sequential()
model1.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model1.add(Dropout(0.1))
model1.add(Dense(30))
model1.add(Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model1, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=25, batch_size=70, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)

print('Test Loss:', score)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:01] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:05] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:05] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:06] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:06] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:09] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:09] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:11] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:11] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:13] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:13] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:30:15] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0063
Test accuracy: 0.006278533488512039


# **Submetering_3**

In [ ]:
df = df[['Sub_metering_3','Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_2', 'Sub_metering_1']]

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

# Creating Spark context
master_url = "'local[8]'"
conf = SparkConf().setAppName("IOT_LSTM_Elephas_App")
conf.setMaster(master_url)
sc = SparkContext.getOrCreate(conf = conf)

model7 = Sequential()
model7.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
model7.add(Dropout(0.1))
model7.add(Dense(1))
model7.compile(loss='mean_squared_error', optimizer='adam')


# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model7, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=40, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)

print('Test Loss:', score)


>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:14] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:17] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:17] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:18] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:18] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:20] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:20] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:21] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:21] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:22] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:22] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:37:24] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0063
Test accuracy: 0.006315270904451609


# **Global Reactive Power**

In [ ]:
df = pd.read_csv('household_power_consumption.txt', sep=';', 
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')

# sns.heatmap(df.isnull(), yticklabels=False, cmap='viridis')

len(df)

import numpy as np
for cols in df.iloc[:,2:].columns:
    print(cols)
    for x in [cols]:
        q75,q25 = np.percentile(df.loc[:,x],[75,25])
        intr_qr = q75-q25
        
        max1 = q75+(1.5*intr_qr)
        min1 = q25-(1.5*intr_qr)
        
        df.loc[df[x] < min1,x] = np.NaN
        df.loc[df[x] > max1,x] = np.NaN

# sns.heatmap(df.isnull(), yticklabels=False, cmap='viridis')    



df = df.fillna(df.mean())
df.isnull().sum()



df = df[['Global_reactive_power', 'Sub_metering_3','Global_active_power',  'Voltage',
       'Global_intensity', 'Sub_metering_2', 'Sub_metering_1']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

# Scalarization
values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd


model2 = Sequential()
model2.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model2.add(Dropout(0.1))
model2.add(Dense(30))
model2.add(Dense(1))
model2.compile(loss='mean_squared_error', optimizer='adam')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model2, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=20, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)


Voltage
Global_intensity
Sub_metering_1
Sub_metering_2
Sub_metering_3
>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
>>> Initialize workers
>>> Distribute load
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:25] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:28] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:28] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:31] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:31] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:33] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:33] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:36] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:36] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:37] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:40:37] "GET /parameters HTTP/1.1" 200 -
INFO:

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0054
Test accuracy: 0.005375169683247805


# **Predict Voltage**

In [1]:
df = df[['Voltage', 'Global_active_power',  'Global_reactive_power', 'Global_intensity', 'Sub_metering_2', 'Sub_metering_1','Sub_metering_3']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

model3 = Sequential()
model3.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model3.add(Dropout(0.1))
model3.add(Dense(1))
model3.compile(loss='mean_squared_error', optimizer='adam')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model3, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=13, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)

NameError: ignored

# **Global Intensity**

In [ ]:
df = df[['Global_intensity', 'Global_active_power',  'Global_reactive_power', 'Voltage', 'Sub_metering_2', 'Sub_metering_1','Sub_metering_3']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd


model4 = Sequential()
model4.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model3.add(Dropout(0.1))
model4.add(Dense(30))
model4.add(Dense(1))
model4.compile(loss='mean_squared_error', optimizer='adam')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model4, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=13, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)

# **Submetering_1**

In [ ]:
df = df[['Sub_metering_1', 'Global_active_power',  'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_2','Sub_metering_3']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

model5 = Sequential()
model5.add(LSTM(200, input_shape=(train_x.shape[1], train_x.shape[2])))
model5.add(Dropout(0.5))
model5.add(Dense(1))
model5.compile(loss='mean_squared_error', optimizer='adam')


# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model5, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=50, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:47:54] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:00] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:00] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:02] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:02] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:04] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:04] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:07] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:07] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:09] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:09] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:48:11] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0032
Test Loss: 0.003233599942177534


# **Submetering_2**

In [ ]:
df = df[['Sub_metering_2', 'Global_active_power',  'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1','Sub_metering_3']]

df_resample = df.resample('h').mean() 
df_resample.shape 

from sklearn.preprocessing import MinMaxScaler

values = df_resample.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

# Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
values = reframed.values
n_train_time = 30000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

model6 = Sequential()
model6.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
# model3.add(Dropout(0.5))
model6.add(Dense(1))
model6.compile(loss='mean_squared_error', optimizer='RMSprop')

# Convert TrainX and TrainY to RDD
rdd = to_simple_rdd(sc, train_x, train_y)

# Create Spark model
spark_model = SparkModel(model5, frequency='epoch', mode='asynchronous', num_workers=1)
spark_model.fit(rdd, epochs=50, batch_size=100, validation_data=(test_x, test_y), verbose=2, shuffle=False)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.master_network.evaluate(test_x, test_y)
print('Test Loss:', score)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:27] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:31] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:31] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:33] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:33] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:35] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:35] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:37] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:37] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:40] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:40] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 11:50:42] "POST /update HTTP/1.1" 200 

>>> Async training complete.
144/144 [==============================] - 1s 2ms/step - loss: 0.0031
Test Loss: 0.003145577386021614
